Install Overpass python pakage

In [42]:
!pip install overpass pymongo


     ---------------------------------------- 1.4/1.4 MB 8.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Using the OSM API we have extracted information about the features in Fredericton. We have used the Overpass and pymongo:

    overpass: Python package that provides a simple way to interact with the OpenStreetMap Overpass API, allowing for the querying and retrieval of OpenStreetMap data.

    pymongo: Python package that provides a driver for MongoDB, allowing for the easy integration of MongoDB databases with Python applications.


The below function take the query as argument and use request API to collect information from OSM for the given query. Then the obtained response is filtered and added to the MongoDB collection through MongoDB client. Finally it returns the total number of records entered in the DB collection. 

In [8]:
import overpass
from pymongo import MongoClient

In [21]:
# set up the MongoDB client
client = MongoClient('mongodb://localhost:27017/')
db = client['my_database']
collection = db['my_collection']

In [22]:

def add_data(query):
    # set up the Overpass API client
    api = overpass.API()

    result = api.get(query)
    count=0

    # extract relevant data from the location result
    for element in result['features']:
        data = {
            'name': element['properties'].get('name', None),
            'type': element['geometry']['type'],
            'coordinate': element['geometry']['coordinates'],
            'cuisine': element['properties'].get('cuisine', None),
            'amenity': element['properties'].get('amenity', None),
            'shop': element['properties'].get('shop', None),
            'address' : element['properties']['addr:street']
            
        }
        

        # insert the data into the MongoDB collection
        collection.insert_one(data)
        count+=1
    return count


    

Call the add_data function with the query as Fredericton. The total records are displayed below as output


In [23]:
# search for Fredericton using the Overpass API
query = """
    node["addr:city"="Fredericton"](around:5000,46.0006,-66.6413);
    out;
"""

print("Total entry in the MongoDB: ", add_data(query))

Total entry in the MongoDB:  1722


In [24]:
collection.find_one({})

{'_id': ObjectId('640fffc78e3b80bd3ebcb04f'),
 'name': 'Atlantic Superstore',
 'type': 'Point',
 'coordinate': [-66.657562, 45.978142],
 'cuisine': None,
 'amenity': None,
 'shop': 'supermarket',
 'address': 'Main Street'}

INSERT

Insert a record

In [25]:
features_new={
    'name':'Simply Fresh',
    'type': 'point',
    'coordinate' : [],
    'cuisine' : 'Indain',
    'amenity' : 'restaurant',
    'shop': 'groceries',
    'address' : 'Regent Street'
}

collection.insert_one(features_new)

In [26]:
for x in collection.find({'name':'Simply Fresh'},{ "_id": 0 }):
  print(x)

{'name': 'Simply Fresh', 'type': 'point', 'coordinate': [], 'cuisine': 'Indain', 'amenity': 'restaurant', 'shop': 'groceries', 'address': 'Regent Street'}


Insert Many

In [27]:
mylist = [
  {  "name": "New Restaraunt",'amenity': 'restaurant', "address": "Highway 37"},
  {  "name": "New Bakery",'amenity': 'Bakery', "address": "Lowstreet 27"},
  {  "name": "New Pastry",'amenity': 'Pastry', "address": "Apple st 652"},
  {  "name": "New Store", 'shop': 'Clothes',"address": "Mountain 21"}
]

x =collection.insert_many(mylist)


In [118]:
print(x.inserted_ids)

[ObjectId('640ff0caf493f80f8ca0be39'), ObjectId('640ff0caf493f80f8ca0be3a'), ObjectId('640ff0caf493f80f8ca0be3b'), ObjectId('640ff0caf493f80f8ca0be3c')]


Find/Select

The Below code returns the name of the features thats with 'New'

In [28]:
for x in collection.find({"name": {"$regex": "^New"}},{ "_id": 0 }):
  print(x)

{'name': 'New Brunswick Public Library Service - York', 'type': 'Point', 'coordinate': [-66.614257, 45.982947], 'cuisine': None, 'amenity': None, 'shop': None, 'address': 'Two Nations Crossing'}
{'name': 'New Brunswick Public Library Service - Provincial Office', 'type': 'Point', 'coordinate': [-66.614265, 45.98235], 'cuisine': None, 'amenity': None, 'shop': None, 'address': 'Two Nations Crossing'}
{'name': 'New Brunswick Public Library Service - York', 'type': 'Point', 'coordinate': [-66.614257, 45.982947], 'cuisine': None, 'amenity': None, 'shop': None, 'address': 'Two Nations Crossing'}
{'name': 'New Brunswick Public Library Service - Provincial Office', 'type': 'Point', 'coordinate': [-66.614265, 45.98235], 'cuisine': None, 'amenity': None, 'shop': None, 'address': 'Two Nations Crossing'}
{'name': 'New Restaraunt', 'amenity': 'restaurant', 'address': 'Highway 37'}
{'name': 'New Bakery', 'amenity': 'Bakery', 'address': 'Lowstreet 27'}
{'name': 'New Pastry', 'amenity': 'Pastry', 'add

The below code gives all the records in the collection without their id.

In [29]:
for x in collection.find({},{ "_id": 0 }):
  print(x)

{'name': 'Atlantic Superstore', 'type': 'Point', 'coordinate': [-66.657562, 45.978142], 'cuisine': None, 'amenity': None, 'shop': 'supermarket', 'address': 'Main Street'}
{'name': 'Subway', 'type': 'Point', 'coordinate': [-66.652035, 45.989185], 'cuisine': 'sandwich', 'amenity': 'fast_food', 'shop': None, 'address': 'Douglas Avenue'}
{'name': 'Subway', 'type': 'Point', 'coordinate': [-66.648713, 45.976973], 'cuisine': 'sandwich', 'amenity': 'fast_food', 'shop': None, 'address': 'Main Street'}
{'name': "St. Mary's Supermarket", 'type': 'Point', 'coordinate': [-66.630694, 45.972062], 'cuisine': None, 'amenity': None, 'shop': 'supermarket', 'address': 'Cliffe Street'}
{'name': 'The Abbey; Red Rover Cider House', 'type': 'Point', 'coordinate': [-66.639913, 45.961448], 'cuisine': None, 'amenity': 'restaurant', 'shop': None, 'address': 'Queen Street'}
{'name': 'Snooty Fox', 'type': 'Point', 'coordinate': [-66.63876, 45.960665], 'cuisine': None, 'amenity': 'pub', 'shop': None, 'address': 'Reg

The below code give the Name, Amenity and Address of the features available in Regent street whose names are Not equal to None.

In [98]:
for i in collection.find({'address':'Regent Street','name':{"$ne": None}},{'_id':0,'name':1, 'amenity':1 ,'address':1 }):
    print(i)

{'name': 'Snooty Fox', 'amenity': 'pub', 'address': 'Regent Street'}
{'name': 'Arom', 'amenity': 'restaurant', 'address': 'Regent Street'}
{'name': 'MidSea Eatery', 'amenity': 'restaurant', 'address': 'Regent Street'}
{'name': "Clipperman's", 'amenity': None, 'address': 'Regent Street'}
{'name': 'Snooty Fox', 'amenity': 'pub', 'address': 'Regent Street'}
{'name': 'Arom', 'amenity': 'restaurant', 'address': 'Regent Street'}
{'name': 'MidSea Eatery', 'amenity': 'restaurant', 'address': 'Regent Street'}
{'name': "Clipperman's", 'amenity': None, 'address': 'Regent Street'}


The below code give the Name and Address of the features whose names are Not equal to None and address startswith T or more

In [107]:
for  y in collection.find({'address':{'$gte': "T"},'name':{"$ne": None}},{'_id':0,'name':1,  'address':1 }):
    print(y)

{'name': 'Luna Pizza', 'address': 'York Street'}
{'name': 'Pita Fresco', 'address': 'York Street'}
{'name': 'Kinsmen Club of Fredericton', 'address': 'Waggoners Lane'}
{'name': "Smoke's Poutinerie", 'address': 'York Street'}
{'name': 'Peter Roberts for Men', 'address': 'York Street'}
{'name': 'Strange Adventures', 'address': 'York Street'}
{'name': 'Urban Almanac General Store', 'address': 'York Street'}
{'name': 'Burger King', 'address': 'Two Nations Crossing'}
{'name': "Grab 'n Go", 'address': 'Two Nations Crossing'}
{'name': 'Dollarama', 'address': 'Two Nations Crossing'}
{'name': 'GNC', 'address': 'Two Nations Crossing'}
{'name': 'New Brunswick Public Library Service - York', 'address': 'Two Nations Crossing'}
{'name': 'New Brunswick Public Library Service - Provincial Office', 'address': 'Two Nations Crossing'}
{'name': "The Children's Academy", 'address': 'Two Nations Crossing'}
{'name': 'Dynamic Health Physiotherapy', 'address': 'Two Nations Crossing'}
{'name': 'The Purrfect Cup

The below code gives the name and address of Restaraunts that serve Pizza in Fredericton in Alphabetical order


In [30]:
for  z in collection.find({'amenity':'restaurant','name':{"$ne": None},'cuisine':'pizza'},{'_id':0,'name':1, 'cuisine':1, 'amenity':1, 'address':1 }).sort('name',1):
    print(z)

{'name': "Gisele's Pizza", 'cuisine': 'pizza', 'amenity': 'restaurant', 'address': 'Two Nations Crossing'}
{'name': "Gisele's Pizza", 'cuisine': 'pizza', 'amenity': 'restaurant', 'address': 'Two Nations Crossing'}
{'name': "Jack's Pizza", 'cuisine': 'pizza', 'amenity': 'restaurant', 'address': 'King Street'}
{'name': "Jack's Pizza", 'cuisine': 'pizza', 'amenity': 'restaurant', 'address': 'King Street'}
{'name': 'Luna Pizza', 'cuisine': 'pizza', 'amenity': 'restaurant', 'address': 'York Street'}
{'name': 'Luna Pizza', 'cuisine': 'pizza', 'amenity': 'restaurant', 'address': 'York Street'}
{'name': 'Luna Pizza', 'cuisine': 'pizza', 'amenity': 'restaurant', 'address': 'York Street'}
{'name': 'Luna Pizza', 'cuisine': 'pizza', 'amenity': 'restaurant', 'address': 'York Street'}
{'name': "Zio's Pizza", 'cuisine': 'pizza', 'amenity': 'restaurant', 'address': 'Main Street'}
{'name': "Zio's Pizza", 'cuisine': 'pizza', 'amenity': 'restaurant', 'address': 'Main Street'}


The below code gives the name of Bakery in Fredericton

In [112]:
for  bakery in collection.find({'shop':'bakery','name':{"$ne": None}},{'_id':0,'name':1, 'shop':1, 'address':1 }):
    print(bakery)

{'name': 'The Happy Baker', 'shop': 'bakery', 'address': 'King Street'}
{'name': 'The Happy Baker', 'shop': 'bakery', 'address': 'King Street'}


UPDATE

The below code update a field of a record

In [125]:
collection.update_one({'name': 'New Store'}, {"$set": {"name": "New Clothe Store"}})

In [126]:
for i in collection.find({'name':'New Clothe Store'},{'_id':0,'name':1, 'shop':1, 'address':1 }):
    print(i)

{'name': 'New Clothe Store', 'shop': 'Clothes', 'address': 'Mountain 21'}


Update Many


In [146]:
collection.update_many({"cuisine": "pizza"}, {"$set": {"rating": 5}})


In [147]:
for i in collection.find({'cuisine':'pizza'},{'_id':0,'name':1, 'rating':1}):
    print(i)

{'name': "Jack's Pizza", 'rating': 5}
{'name': 'Luna Pizza', 'rating': 5}
{'name': "Domino's", 'rating': 5}
{'name': "Zio's Pizza", 'rating': 5}
{'name': "Papa John's", 'rating': 5}
{'name': "Gisele's Pizza", 'rating': 5}
{'name': 'Luna Pizza', 'rating': 5}
{'name': "Jack's Pizza", 'rating': 5}
{'name': 'Luna Pizza', 'rating': 5}
{'name': "Domino's", 'rating': 5}
{'name': "Zio's Pizza", 'rating': 5}
{'name': "Papa John's", 'rating': 5}
{'name': "Gisele's Pizza", 'rating': 5}
{'name': 'Luna Pizza', 'rating': 5}


DELETE

Delete one record from collection


In [149]:
for i in collection.find({'name':'New Clothe Store'},{'_id':0,'name':1}):
    print(i)

{'name': 'New Clothe Store'}


In [150]:
collection.delete_one({"name": 'New Clothe Store'})

In [151]:
for i in collection.find({'name':'New Clothe Store'},{'_id':0,'name':1}):
    print(i)

Delete Many 

In [153]:
for i in collection.find({'cuisine':'thai'},{'_id':0,'name':1, 'rating':1}):
    print(i)

{'name': 'Thai Spice Restaurant', 'rating': 5}
{'name': 'Thai Spice Restaurant', 'rating': 5}


In [154]:
collection.delete_many({'cuisine':'thai'})

In [155]:
for i in collection.find({'cuisine':'thai'},{'_id':0,'name':1, 'rating':1}):
    print(i)